In [1]:
%load_ext iminizinc
%load_ext autoreload
%autoreload 2

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
# import ipywidgets as widgets
from ipywidgets import *
from IPython.display import display
import IPython
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

from datetime import datetime
# import talib
import pandas_ta as ta
# from talib.abstract import *
from math import *
from collections import OrderedDict
# import vectorbt as vbt    
import json
from decimal import *
getcontext().prec = 6
import seaborn as sns
from scipy.signal import find_peaks, peak_widths


import sys, os
sys.path.append(os.path.abspath('../..'))
from research.pkg.lib import *


# import mplfinance as mpf
%matplotlib inline

mpl.rcParams.update({'font.size': 5, 'lines.linewidth': 0.5, 'figure.dpi': 200, 'lines.markersize': 4})
# plt.rcParams['lines.linewidth'] = 0.5 



<IPython.core.display.Javascript object>

MiniZinc to FlatZinc converter, version 2.7.6, build 905165378
Copyright (C) 2014-2023 Monash University, NICTA, Data61


In [2]:
exchange = 'kucoin'
pair = 'BTC_USDT'
timeframe = '4h'

odf = pd.read_json(f'../../freq-user-data/data/{exchange}/futures/{pair}-{timeframe}-futures.json').dropna()
odf.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']

odf['date'] = pd.to_datetime(odf['timestamp'], unit='ms', utc=False)
# df.index = df['time']
# df.set_index('time', drop=True, inplace=True)
odf['idate'] = odf.date.dt.strftime('%Y%m%d')
odf.set_index(pd.DatetimeIndex(odf["date"]), inplace=True, drop=True)
# df = df[['time', 'symbol', 'source', 'resolution', 'open', 'high', 'low', 'close', 'volume']]
# df.to_csv (r'./data/binance/BTC_USDT-5m.csv', index = None)
# df.set_index('time')
odf = odf.sort_index()

odf.head(5)

,timestamp,open,high,low,close,volume,date,idate
date,,,,,,,,
2020-01-01 00:00:00,1577836800000,7196.2,7243.2,7176.1,7225.4,62.743859,2020-01-01 00:00:00,20200101
2020-01-01 04:00:00,1577851200000,7225.4,7233.8,7203.1,7209.5,36.248625,2020-01-01 04:00:00,20200101
2020-01-01 08:00:00,1577865600000,7209.4,7232.0,7183.0,7195.6,52.545595,2020-01-01 08:00:00,20200101
2020-01-01 12:00:00,1577880000000,7195.7,7251.8,7195.6,7231.4,69.704852,2020-01-01 12:00:00,20200101
2020-01-01 16:00:00,1577894400000,7231.5,7248.0,7217.1,7229.0,60.889127,2020-01-01 16:00:00,20200101


In [38]:
h_columns = ['open', 'high', 'low', 'close', 'volume']
hdf = odf[h_columns].resample('1D').agg(
    OrderedDict([
        ('open', 'first'),
        ('high', 'max'),
        ('low', 'min'),
        ('close', 'last'),
        ('volume', 'sum'),
    ])
)
hdf.head(5)

,open,high,low,close,volume
date,,,,,
2020-01-01,7196.2,7251.8,7176.1,7200.2,370.594061
2020-01-02,7200.3,7210.6,6933.3,6964.8,834.203799
2020-01-03,6964.8,7397.7,6867.0,7346.2,1325.683236
2020-01-04,7344.6,7399.0,7271.5,7355.3,695.820296
2020-01-05,7355.9,7490.0,7320.7,7359.9,948.491582


In [48]:
gwdf.index.date[0]

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [49]:
gwdf = None
def get_indicators(wdf, pkdist, pkprom):
    global gwdf
    gwdf = wdf
    whdf = hdf.loc[wdf.index.date[0]:wdf.index.date[-1]]

    upeaks, upeak_props = find_peaks(whdf.close, prominence=1, distance=pkdist)
    dpeaks, dpeak_props = find_peaks(-whdf.close, prominence=1, distance=pkdist)
    # print('upeaks: ', upeaks)
    # print('wdf:', wdf)
    # results_half = peak_widths(wdf.close, peaks, rel_height=0.5)
    # results_full = peak_widths(wdf.close, peaks, rel_height=1)
    
    upeaks_idxs = whdf.index[upeaks]
    dpeaks_idxs = whdf.index[dpeaks]

    return upeaks_idxs,dpeaks_idxs, gwdf

def get_signals(wdf, pkdist, pkprom):
    sigdf = None
    # upeaks, upeak_props = find_peaks(wdf.close, distance=pkdist)
    # dpeaks, dpeak_props = find_peaks(-wdf.close, distance=pkdist)

    # # results_half = peak_widths(wdf.close, peaks, rel_height=0.5)
    # # results_full = peak_widths(wdf.close, peaks, rel_height=1)
    
    # upeaks_idxs = wdf.index[upeaks]
    # dpeaks_idxs = wdf.index[dpeaks]

    # close_peaks = wdf.loc[peaks_idx].close
    return None #upeaks_idxs,dpeaks_idxs

In [50]:

### Ranges
# nest_asyncio.apply()

strategy = 'swings'
strategy_params_json = f'./par-{strategy}.json'

strategy_params = {
    "pkdist": {"wdg": IntSlider(value=7,min=1,max=100,step=1,description='PKDist',continuous_update=False)},
    "pkprom": {"wdg": FloatRangeSlider(value=(0.05,1),min=0,max=1,step=0.005,description='PKProm',continuous_update=False)},
}


def draw_func(ax_dict, wdf, indicators, signals):
    axa = ax_dict['A']
    upeaks_idx, dpeaks_idx = indicators
    # print('indicators: ', indicators)
    # print(wdf.close[upeaks_idx])
    axa.scatter(upeaks_idx, wdf.close[upeaks_idx], s=50, c='r', alpha=0.2)
    axa.scatter(dpeaks_idx, wdf.close[dpeaks_idx], s=50, c='g', alpha=0.2)
    # axa.plot(dpeaks_idx, wdf.close[upeaks_idx], "^")
    

fu_printer(odf, strategy, strategy_params = strategy_params, draw_func = draw_func, indicator_func=get_indicators, signal_func=get_signals)

#     ax1.plot(peaks_idx, close_peaks, "^", c='m', markersize=2, alpha=0.5)
#     ax1.vlines(x=peaks_idx, ymin=close_peaks - peak_props["prominences"],
#            ymax = close_peaks, color = "C1", lw=0.2)
#     ax1.hlines(results_full[1:][0], xmin=wdf.index[results_full[1:][1].astype(int)], xmax=wdf.index[results_full[1:][2].astype(int)], color="C1", lw=0.3)
#     ax1.hlines(results_half[1:][0], xmin=wdf.index[results_half[1:][1].astype(int)], xmax=wdf.index[results_half[1:][2].astype(int)], color="C2", lw=0.3)
#     # ax1.hlines(y=(close_peaks - peak_props["prominences"]), xmin=wdf.index[peak_props["left_bases"]],
#     #        xmax=wdf.index[peak_props["right_bases"]], color = "C1")




Output()

In [76]:

peaks, peak_props = find_peaks(wdf.close, prominence=1, distance=10)
# wdf.iloc[peaks].close

results_half = peak_widths(wdf.close, peaks, rel_height=0.5)
# np.shape(results_half[1:])
results_half[1:][1].astype(int)
# fig, (ax1,ax2, ax3) = plt.subplots(3, 1, height_ratios=[3,1,1], sharex=True);

# plot_candles(wdf, ax=ax1) if candles else ax1.plot(wdf.close, lw=0.3, c='b')
# plt.plot(peaks, wdf.iloc[peaks].close, "x")

array([  1,   9,  24,  35,  49,  61,  81,  91, 101, 112, 127, 142, 159,
       172, 189, 202, 218, 229, 245, 240, 266, 279, 302, 316, 320, 338,
       347, 358, 366, 383, 381, 406, 418, 428, 439, 450, 463, 480, 501])

In [56]:
tuple([1,2])

(1, 2)